<a href="https://colab.research.google.com/github/JoergNeumann/GenAI/blob/main/Agent_SDK_Demo_en.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Agent SDK Demo

Demonstrates the use of OpenAI's Agent SDK.

**Setup**

In [1]:
!pip install -qU openai-agents==0.0.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 9.2 MB/s eta 0:00:00


In [2]:
import os
import json
from agents import set_tracing_export_api_key

# Read OpenAI API Key from Colab Secret and create OpenAI Client
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

set_tracing_export_api_key(os.environ.get('OPENAI_API_KEY'))

# Define Agent

In [3]:
from agents import Agent, Runner

agent = Agent(
    name="Assistant",
    instructions="You are a helpful assistant.",
    model="gpt-4o-mini",
)

# Run Agent

In [4]:
result = await Runner.run(
    starting_agent=agent,
    input="Create a simple lunch menu."
)
output = result.final_output

# Output Markdown
from IPython.display import Markdown
display(Markdown(output))

Sure! Here’s a simple lunch menu:

### Lunch Menu

#### **1. Sandwiches**
- **Turkey & Avocado Club**  
  Sliced turkey, fresh avocado, lettuce, tomato, and mayo on whole grain bread.

- **Caprese Panini**  
  Fresh mozzarella, tomatoes, basil, and balsamic glaze pressed on ciabatta.

#### **2. Salads**
- **Caesar Salad**  
  Crisp romaine, parmesan cheese, croutons, and creamy Caesar dressing.

- **Quinoa & Black Bean Salad**  
  Quinoa, black beans, corn, red bell pepper, cilantro, and lime vinaigrette.

#### **3. Wraps**
- **Hummus & Veggie Wrap**  
  Hummus, mixed veggies, and spinach in a whole wheat tortilla.

- **Buffalo Chicken Wrap**  
  Spicy grilled chicken, lettuce, celery, and ranch dressing in a flour tortilla.

#### **4. Soups**
- **Tomato Basil Soup**  
  Creamy tomato soup with fresh basil served with a side of crusty bread.

- **Chicken Noodle Soup**  
  Classic chicken soup with vegetables and egg noodles.

#### **5. Sides**
- **Fresh Fruit Cup**  
  A mix of seasonal fruits.

- **Chips**  
  Choice of regular or spicy tortilla chips.

#### **6. Beverages**
- **Iced Tea**  
  Freshly brewed, sweetened or unsweetened.

- **Sparkling Water**  
  Refreshing sparkling mineral water.

Feel free to choose any combination for a delicious and satisfying lunch!

# Streaming
The output can also be streamed.

In [5]:
from openai.types.responses import ResponseTextDeltaEvent

response = Runner.run_streamed(
    starting_agent=agent,
    input="Explain the basics of generative AI"
)

async for event in response.stream_events():
    if event.type == "raw_response_event" and \
        isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Generative AI refers to a class of artificial intelligence models designed to generate new content based on existing data. Here are some key concepts to understand the basics:

### 1. **Definition and Purpose**
- **Generative AI** generates text, images, music, or other types of data. Unlike discriminative models, which classify or analyze input data, generative models create new instances that resemble the training data.

### 2. **Types of Generative Models**
- **Generative Adversarial Networks (GANs)**: Consist of two neural networks—a generator and a discriminator—that compete against each other. The generator creates data, while the discriminator evaluates its authenticity.
- **Variational Autoencoders (VAEs)**: These encode input data into a compressed representation and then decode it to generate new data, allowing for smooth interpolation between different data points.
- **Transformers**: Models like GPT (Generative Pre-trained Transformer) generate text by predicting the next w

# Integrate Functions

In [6]:
shopping_list = []

from agents import function_tool

@function_tool
def add_to_shopping_list(article: str):
    shopping_list.append(article)
    print(f"### add_to_shopping_list: '{article}' was added to the shopping list. ###")
    #print("Current shopping list:", shopping_list)


Now we can register the function as a tool with the agent...

In [7]:
agent = Agent(
    name="Assistant",
    instructions=("You are a helpful assistant."),
    model="gpt-4o-mini",
    tools=[add_to_shopping_list]
)

... and start the query

In [8]:
response = await Runner.run(
    starting_agent=agent,
    input="Add eggs to the shopping list!"
)
print(response.final_output)


### add_to_shopping_list: 'eggs' was added to the shopping list. ###
I've added eggs to the shopping list! Let me know if you need anything else.


# Integrate Tool
WebSearch is a pre-built tool from OpenAI. It allows using web search to complete the task.

In [9]:
from agents import WebSearchTool

web_agent = Agent(
    name="Web Assistant",
    instructions="You are an assistant who retrieves current news from the internet.",
    output_type=str,
    tools=[WebSearchTool()],
)
output = await Runner.run(
    starting_agent=web_agent,
    input="Find the results of the last matchday (32nd matchday, 2025) of the German Football Bundesliga.",
)
from IPython.display import Markdown
display(Markdown(output.final_output))

The 32nd matchday of the 2024/2025 German Bundesliga season took place from May 2 to May 4, 2025. Here are the results:

- **May 2, 2025 (Friday):**
  - 1. FC Heidenheim 1846 0 – 0 VfL Bochum

- **May 3, 2025 (Saturday):**
  - Borussia Dortmund 4 – 0 VfL Wolfsburg
  - RB Leipzig 3 – 3 Bayern Munich
  - Borussia Mönchengladbach 4 – 4 TSG 1899 Hoffenheim
  - 1. FC Union Berlin 2 – 2 Werder Bremen
  - FC St. Pauli 0 – 1 VfB Stuttgart

- **May 4, 2025 (Sunday):**
  - FC Augsburg 1 – 3 Holstein Kiel
  - SC Freiburg 2 – 2 Bayer 04 Leverkusen
  - 1. FSV Mainz 05 1 – 1 Eintracht Frankfurt

Notably, Bayern Munich secured the Bundesliga title on May 4, 2025, following Bayer Leverkusen's 2–2 draw with SC Freiburg. ([en.wikipedia.org](https://en.wikipedia.org/wiki/2024%E2%80%9325_Bundesliga?utm_source=openai)) 

# Conversation

In [10]:
# Call agent with an instruction
result = await Runner.run(
    starting_agent=agent,
    input="Calculate the square root of 10."
)

print(f'1st pass: {result.final_output}')
print(json.dumps(result.to_input_list(), indent=4, ensure_ascii=False))

# Call agent again with the previous result and an additional query
result = await Runner.run(
    starting_agent=agent,
    input=result.to_input_list() + [
        {"role": "user", "content": "Generate a Python function for the calculation."}
    ]
)
print(f'2nd pass: {result.final_output}')

1st pass: The square root of 10 is approximately 3.1623.
[
    {
        "content": "Calculate the square root of 10.",
        "role": "user"
    },
    {
        "id": "msg_686bdf60b984819ab4bc5e2483f14aef088273285b90807c",
        "content": [
            {
                "annotations": [],
                "text": "The square root of 10 is approximately 3.1623.",
                "type": "output_text",
                "logprobs": []
            }
        ],
        "role": "assistant",
        "status": "completed",
        "type": "message"
    }
]
2nd pass: Here’s a simple Python function to calculate the square root of a number:

```python
import math

def calculate_square_root(number):
    if number < 0:
        raise ValueError("Cannot calculate the square root of a negative number.")
    return math.sqrt(number)

# Example usage:
result = calculate_square_root(10)
print(result)  # Output: 3.1622776601683795
```

You can replace `10` in the example usage with any number you wan

# Handoffs
Handoffs refer to the delegation of control from one agent to the next. This allows for flexible decisions on which agent should handle a task based on the prompt.

In [11]:
@function_tool
def record_travel_costs(amount: float):
    return "Travel costs have been recorded."

@function_tool
def record_fuel_costs(amount: float):
    return "Fuel costs have been recorded."

@function_tool
def record_expenses(amount: float):
    return "Expenses have been recorded."

travel_cost_agent = Agent(
    name="Travel Cost Agent",
    instructions="You are an accountant and process travel expenses. Use the provided tool for this.",
    tools=[record_travel_costs],
)
fuel_cost_agent = Agent(
    name="Fuel Cost Agent",
    instructions="You are an accountant and process fuel expenses. Use the provided tool for this.",
    tools=[record_fuel_costs],
)
expenses_agent = Agent(
    name="Expenses Agent",
    instructions="You are an accountant and process expense reports. Use the provided tool for this.",
    tools=[record_expenses],
)

triage_agent = Agent(
    name="Triage Agent",
    instructions="You route the user to a suitable agent. Please do not ask follow-up questions to the user.",
    handoffs=[fuel_cost_agent, travel_cost_agent, expenses_agent]
)

user_query = "Please record my travel expenses of €122."
#user_query = "Please record my fuel costs of €54."
#user_query = "Please record my expenses of €67."
print("User:", user_query)

output = Runner.run_streamed(
    starting_agent=triage_agent,
    input=user_query,
)
from openai.types.responses import (
    ResponseFunctionCallArgumentsDeltaEvent,
    ResponseCreatedEvent,
)
async for event in output.stream_events():
    if event.type == "raw_response_event":
        if isinstance(event.data, ResponseFunctionCallArgumentsDeltaEvent):
            print(event.data.delta, end="", flush=True)
        elif isinstance(event.data, ResponseTextDeltaEvent):
            print(event.data.delta, end="", flush=True)
    elif event.type == "agent_updated_stream_event":
        print(f"> Current Agent: {event.new_agent.name}")
    elif event.type == "run_item_stream_event":
        if event.name == "tool_called":
            print()
            print(f"> Tool called, Name: {event.item.raw_item.name}")
            print(f"> Tool called, Args: {event.item.raw_item.arguments}")
        elif event.name == "tool_output":
            print(f"> Tool Output: {event.item.raw_item['output']}")

User: Please record my travel expenses of €122.
> Current Agent: Triage Agent
{}> Current Agent: Expenses Agent
{"amount":122}
> Tool called, Name: record_expenses
> Tool called, Args: {"amount":122}
> Tool Output: Expenses have been recorded.
Your travel expenses of €122 have been recorded successfully.

# Agents as Tools
Multiple agents can also share a large task. This involves breaking down the task into subtasks and assigning them to the agents. The main agent receives the overall task and automatically forwards it to the subordinate agents. For this purpose, they are assigned to the main agent as tools.

In [12]:
from agents import Agent, Runner, ItemHelpers, MessageOutputItem
from openai.types.responses import (
    ResponseFunctionCallArgumentsDeltaEvent,
    ResponseCreatedEvent,
)
dessert_agent = Agent(
    name="Dessert Chef",
    instructions="You are a chef specializing in desserts. Always start your output with the word '[DESSERT]'.",
)
main_course_agent = Agent(
    name="Main Course Chef",
    instructions="You are a chef specializing in main courses. Always start your output with the word '[MAIN_COURSE]'.",
)
appetizer_agent = Agent(
    name="Appetizer Chef",
    instructions="You are a chef specializing in appetizers. Always start your output with the word '[APPETIZER]'.",
)

recipe_agent = Agent(
    name="Recipe Assistant",
    instructions="""
    You are an assistant who helps create recipes. However, you only provide the idea for appetizer, main course, and dessert.
    You use the tools provided to create the recipes for each course.
    If asked for multiple courses, you call the corresponding tools sequentially.
    You never generate a recipe independently, but always use the provided tools.
    """,
    tools=[
        appetizer_agent.as_tool(
            tool_name="Appetizer",
            tool_description="Provides the recipe for the appetizer.",
        ),
        main_course_agent.as_tool(
            tool_name="MainCourse",
            tool_description="Provides the recipe for the main course.",
        ),
        dessert_agent.as_tool(
            tool_name="Dessert",
            tool_description="Provides the recipe for the dessert.",
        ),
    ],
)

output = Runner.run_streamed(
    starting_agent=recipe_agent,
    input="Generate a lunch menu with an appetizer, main course, and dessert. Please provide the complete recipes for each.",
)

async for event in output.stream_events():
    if event.type == "raw_response_event":
        if isinstance(event.data, ResponseFunctionCallArgumentsDeltaEvent):
            print(event.data.delta, end="", flush=True)
        elif isinstance(event.data, ResponseTextDeltaEvent):
            print(event.data.delta, end="", flush=True)
    elif event.type == "agent_updated_stream_event":
        print(f"> Current Agent: {event.new_agent.name}")
    elif event.type == "run_item_stream_event":
        if event.name == "tool_called":
            print()
            print(f"> Tool called, Name: {event.item.raw_item.name}")
            print(f"> Tool called, Args: {event.item.raw_item.arguments}")
        elif event.name == "tool_output":
            print(f"> Tool Output: {event.item.raw_item['output']}")

print(f"""
    \n\n---------------------------\n\n
    Total result:\n{output.final_output}
""")


> Current Agent: Recipe Assistant
{"input":"lunch"}{"input":"lunch"}{"input":"lunch"}
> Tool called, Name: Appetizer
> Tool called, Args: {"input":"lunch"}

> Tool called, Name: MainCourse
> Tool called, Args: {"input":"lunch"}

> Tool called, Name: Dessert
> Tool called, Args: {"input":"lunch"}
> Tool Output: [APPETIZER] Looking for the perfect starter to your lunch? How about some crispy bruschetta topped with fresh tomatoes, basil, and a hint of garlic on toasted baguette slices? It's a refreshing and tasty way to kick off your meal!
> Tool Output: [MAIN_COURSE] How about preparing a savory Chicken Alfredo Pasta? It's a classic choice for lunch, combining tender chicken pieces with creamy Alfredo sauce and perfectly cooked fettuccine. Pair it with garlic bread and a side salad for a complete meal. Enjoy!
> Tool Output: [DESSERT] After a delightful lunch, how about a refreshing dessert suggestion? A zesty lemon sorbet would cleanse the palate beautifully, or perhaps a classic tiramis

# Guardrails

Guardrails can be defined for inputs and outputs. They can be used to filter out undesirable content from communication.

In [14]:
import asyncio
from agents import (
    Agent,
    Runner,
    GuardrailFunctionOutput,
    RunContextWrapper,
    input_guardrail,
    InputGuardrailTripwireTriggered
)
from pydantic import BaseModel

# Structure for the guardrail check
class GuardrailCheck(BaseModel):
    is_triggered: bool
    reason: str

# Agent for the guardrail check
politics_agent = Agent(
    name="Politics Checker",
    instructions="Checks if the user is asking about political topics.",
    output_type=GuardrailCheck,
)

# Guardrail function that calls the agent
@input_guardrail
async def politics_guardrail(
    ctx: RunContextWrapper[None],
    agent: Agent,
    input: str,
) -> GuardrailFunctionOutput:
    response = await Runner.run(starting_agent=politics_agent, input=input)
    return GuardrailFunctionOutput(
        output_info=response.final_output,
        tripwire_triggered=response.final_output.is_triggered,
    )

# Parent agent that uses the guardrail function
agent = Agent(
    name="Assistant",
    instructions=("You are a helpful assistant."
    ),
    model="gpt-4o-mini",
    input_guardrails=[politics_guardrail],
)

# Test query
try:
    query = "What do you think about the new federal government?"
    result = await Runner.run(starting_agent=agent, input=query)
    print(result.final_output)
except InputGuardrailTripwireTriggered as e:
    print(f"Guardrail triggered. Reason: {e.guardrail_result.output.output_info.reason}")


Guardrail triggered. Reason: The inquiry pertains to the evaluation of a newly established federal government, which is a political topic.
